In [1]:
import os
from pathlib import Path
from typing import Optional

import fastmri
import h5py
import matplotlib.pyplot as plt
import numpy as np
import torch
from data_utils import *
from datasets import *
from fastmri.data.transforms import tensor_to_complex_np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from torch.utils.data import DataLoader, TensorDataset

from model import *
from torch.optim import SGD, Adam, AdamW
from train_utils import *

In [12]:
model_checkpoint = '/scratch_net/ken/mcrespo/proj_marina/logs/multivol/2024-11-13_12h31m52s/checkpoints/epoch_0999.pt'  # TODO: SET (OR LEAVE COMMENTED).
files = '/itet-stor/mcrespo/bmicdatasets-originals/Originals/fastMRI/brain/multicoil_train/'


dataset = KCoordDataset(files, n_volumes=3, n_slices=3, with_mask=False)
print(len(dataset))
# loader_config = config["dataloader"]
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True, collate_fn=collate_fn, pin_memory=PIN_MEMORY, worker_init_fn=seed_worker, generator=RS_TORCH)
# dataloader = DataLoader(
#     dataset,
#     batch_size=120_000,
#     num_workers=0,
#     shuffle=True,
#     pin_memory=False,
#     )

14534400


In [11]:
30471680//360_000

84

In [5]:
gamma = 0.1
sigma = 0.01
lr = 5.e-6
embedding_dim = 512

OPTIMIZER_CLASSES = {
    "Adam": Adam,
    "AdamW": AdamW,
    "SGD": SGD,
}

LOSS_CLASSES = {
    "MAE": MAELoss,
    "DMAE": DMAELoss,
    "MSE": MSELoss,
    "MSEDist": MSEDistLoss,
    "HDR": HDRLoss,
    "LogL2": LogL2Loss,
    "MSEL2": MSEL2Loss,
}


model = Siren(hidden_dim=512, levels=5, embedding_dim=512, n_min=45, n_layers=6, out_dim=2, dropout_rate=0)
# Load checkpoint.
model_state_dict = torch.load(model_checkpoint)["model_state_dict"]
model.load_state_dict(model_state_dict)
print("Checkpoint loaded successfully.")

# Only embeddings are optimized.
for param in model.parameters():
    param.requires_grad = False



embeddings = torch.nn.Embedding(
    len(dataset.metadata), embedding_dim
)
torch.nn.init.normal_(
    embeddings.weight.data, 0.0, sigma
)
optimizer = OPTIMIZER_CLASSES["Adam"](
    embeddings.parameters(), lr
)

loss_fn = LOSS_CLASSES["MSEL2"](gamma, sigma)

Checkpoint loaded successfully.


/tmp/ipykernel_3295987/3476506335.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_checkpoint)["model_state_dict"]


In [6]:
print(f"model {model}")
print(f"loss {loss_fn}")
print(f"optimizer {optimizer}")
# print(f"scheduler {scheduler}")
# print(config)
print(f"Number of steps per epoch: {len(dataloader)}")


model Siren(
  (embed_fn): hash_encoder(
    (embeddings): ModuleList(
      (0): Embedding(2500, 2)
      (1-4): 4 x Embedding(4096, 2)
    )
  )
  (sine_layers): ModuleList(
    (0): SineLayer(
      (linear): Linear(in_features=524, out_features=512, bias=True)
    )
    (1-2): 2 x SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
    (3): SineLayer(
      (linear): Linear(in_features=1036, out_features=512, bias=True)
    )
    (4-5): 2 x SineLayer(
      (linear): Linear(in_features=512, out_features=512, bias=True)
    )
  )
  (output_layer): Linear(in_features=512, out_features=2, bias=True)
)
loss <train_utils.MSEL2Loss object at 0x7f4c73c49760>
optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5e-06
    maximize: False
    weight_decay: 0
)
Number of steps per epoch: 122


In [8]:
## START TRAINING FROM CHECKPOINT


NameError: name 'config' is not defined